In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [3]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, reset_output, output_file

In [4]:
# Bokeh version
bokeh.__version__

'2.1.1'

In [5]:
# reading data
df = pd.read_csv('data/covid_19_data.csv')

In [6]:
#Checking null values
df.isna().sum()   # or df.isnull().sum()

SNo                    0
ObservationDate        0
Province/State     10001
Country/Region         0
Last Update            0
Confirmed              0
Deaths                 0
Recovered              0
dtype: int64

In [63]:
# creating required data
#pd.set_option('display.max_rows', 1000)
total_df = df.groupby(by='Country/Region')['Confirmed'].max().to_frame().reset_index()
total_df = total_df[total_df['Confirmed']>3000].reset_index(drop=True)
total_df.columns = ['Country', 'Confirmed']

In [8]:
# top_10 countries by count
top_10= total_df.sort_values(by='Confirmed', ascending=False)[:10].reset_index(drop=True)

In [9]:
# shuffling dataframe
top_10 = sklearn.utils.shuffle(top_10).reset_index(drop=True)
# top_10.columns = ['Country', 'Confirmed']

In [10]:
# output file in jupyter
output_notebook()

Loading BokehJS ...

In [11]:
from bokeh.models import ColumnDataSource

data = {'x_values': [1, 2, 3, 4, 5],
        'y_values': [6, 7, 2, 3, 6]}

column_data_source = ColumnDataSource(data)

### Line Plot

In [12]:
from bokeh.models import HoverTool

line_plot = figure()
line_plot.line('x_values','y_values', source=column_data_source)

#line_plot.line(total_df['Country/Region'], total_df['Confirmed'], legend_label='confirmed')

line_plot.add_tools(HoverTool(tooltips=[('x_values','@x_values')]))
show(line_plot)

In [13]:
# multiple line chart

# data
x = np.arange(1, 11, 1)
y1 = np.random.rand(10)
y2 = np.random.rand(10)
y3 = np.random.rand(10)

# plot
multi_line = figure()
multi_line.line(x, y1, color='blue', legend_label='y1', )
multi_line.line(x, y2, color='red', legend_label='y2')
multi_line.line(x, y3, color='yellow', legend_label='y3')

multi_line.add_tools(HoverTool(tooltips=[('x', '@x'), ('y', '@y')]))

show(multi_line)

### Bar Plot

In [14]:
# simple bar chart

top_10_col = ColumnDataSource(top_10)

#output_file("bars.html")
#reset_output()
#output_notebook()

bar_plot = figure(x_range=top_10['Country'])
#bar_plot.vbar('x_values','y_values', source=column_data_source)

bar_plot.vbar(x='Country', top='Confirmed', source=top_10_col, width=0.5)

#bar_plot.vbar(x=top_10['Country/Region'], top=top_10['Confirmed'], width=0.5)

bar_plot.add_tools(HoverTool(tooltips=[("Country", "@Country"),('Count','@Confirmed')]))

show(bar_plot)


In [15]:
# horizontal stacked bar chart

df = pd.DataFrame({'y': [1, 2, 3, 4, 5],
                               'x1': [1, 2, 4, 3, 4],
                               'x2': [1, 4, 2, 2, 3]})

p = figure(plot_width=600, plot_height=300, title='stacked bar chart')
p.hbar_stack(['x1', 'x2'], y='y', source=df, color=['blue', 'red'], width=20)

# p.add_tools(HoverTool(tooltips=[('x','@x1'),('y', '@y')]))

show(p)

In [35]:
from bokeh.transform import dodge

categories = ['category1', 'category2', 'category3']
grouped_bar_df = pd.DataFrame({'categories' : categories,
                               '2015': [2, 1, 4],
                               '2016': [5, 3, 3],
                               '2017': [3, 2, 4]})
grouped_bar = figure(x_range=categories, plot_height=250)

#dodge
dodge1 = dodge('categories', 0.25, range=grouped_bar.x_range)
dodge2 = dodge('categories', 0, range=grouped_bar.x_range)
dodge3 = dodge('categories', -0.25, range=grouped_bar.x_range)

grouped_bar.vbar(x=dodge1, top='2015', source=grouped_bar_df, width=0.2, legend_label='2015', color='red')
grouped_bar.vbar(x=dodge2, top='2016', source=grouped_bar_df, width=0.2, legend_label='2016', color='blue')
grouped_bar.vbar(x=dodge3, top='2017', source=grouped_bar_df, width=0.2, legend_label='2017', color='yellow')

#format legend
grouped_bar.legend.location = 'top_center'
grouped_bar.legend.orientation = 'horizontal'

# adding tools
grouped_bar.add_tools(HoverTool(tooltips=[('category', '@categories'), ]))

show(grouped_bar)

### Stacked area chart

In [39]:
df = pd.DataFrame({'x': [1, 2, 3, 4, 5],
                                'y1': [1, 2, 4, 3, 4],
                                'y2': [1, 4, 2, 2, 3],
                  'label': ['a', 'b', 'c', 'd', 'e']})
area_chart = figure(plot_height=300)
area_chart.varea_stack(['y1', 'y2'], x='x', color=['blue', 'red'], source=df)

# area_chart.add_tools(HoverTool(tooltips=[ ('value','@y')]))

show(area_chart)

### Getting source code of class
import inspect

inspect.getsource(bokeh.plotting.Figure).split('    ')

### Scatter plot

In [66]:
total_df['conf+100'] = total_df['Confirmed']+100  # adding new column
df = total_df.reset_index()
countries = total_df['Confirmed'].to_list()
scatter = figure(plot_height=300, title='Confirmed cases plot')
scatter.circle('index', 'Confirmed', source=df, size=15, color='green', legend_label='total cases')

scatter.add_tools(HoverTool(tooltips=[('country','@Country')]))

scatter.legend.location='top_left'

show(scatter)

In [95]:
# creating class
df['class_'] = np.random.randint(low=0,high=3,size=len(df['index']))
df['class_'] = df.class_.replace(to_replace=[0,1,2], value=['a', 'b', 'c'])

In [97]:
# scatter plot for clustering
from bokeh.transform import factor_cmap, factor_mark

class_ = ['a', 'b', 'c']
markers = ['hex', 'cross', 'triangle']

scatter_sub = figure()

scatter_sub.scatter(x='index',
                   y='Confirmed',
                   source=df,
                   legend_label='class_',
                   size=15,
                   fill_alpha=0.5,
                    color=factor_cmap(field_name='class_', palette='Dark2_3', factors=class_),
                               marker=factor_mark(field_name='class_', markers=markers, factors=class_)
                   )
show(scatter_sub)

### Subplots

In [71]:
from bokeh.layouts import gridplot

grid_1 = gridplot([scatter, p, area_chart, grouped_bar], ncols=2 )
show(grid_1)

### Linked plot

Interactive operations can be performed on multiple charts

In [132]:
from bokeh.layouts import gridplot

linked_data_x = np.arange(10)
linked_data_y = np.random.rand(10)

# linked plot 1
linked_plot1 = figure(width=250, height=250, tools="")
linked_plot1.circle(linked_data_x, linked_data_y)

# create new plots and share both ranges
linked_plot2 = figure(width=250, height=250, x_range=linked_plot1.x_range, y_range=linked_plot1.y_range, tools="")
linked_plot2.line(linked_data_x, linked_data_y)

linked_plot3 = figure(width=250, height=250, x_range=linked_plot1.x_range, y_range=linked_plot1.y_range, tools="")
linked_plot3.vbar(linked_data_x, top=linked_data_y, width=0.5)

# the subplots in a gridplot
linked_gridplot = gridplot([[linked_plot1, linked_plot2, linked_plot3]], toolbar_options=dict(logo=None)) # no logo

# show the results
show(linked_gridplot)

### Labels and Annotations

In [102]:
from bokeh.models.annotations import Label, LabelSet, Arrow
from bokeh.models.arrow_heads import NormalHead

# data
fig_with_label_data = ColumnDataSource({'x': np.arange(10), 
                                        'y': [4, 7, 5, 5, 9, 2, 3, 4, 3, 4]})

# plot
fig_with_label = figure()
fig_with_label.line(x='x', y='y', source=fig_with_label_data)

# add label
label = Label(x=4, y=9, x_offset=10, text='Higest Point', text_baseline='middle')
fig_with_label.add_layout(label)

# add multiple labels
labels = LabelSet(x='x', y='y', y_offset=10, text='y', level='glyph', source=fig_with_label_data)
fig_with_label.add_layout(labels)

# arrow annotation
fig_with_label.add_layout(Arrow(end=NormalHead(fill_color='green'), x_start=5, y_start=7.5, x_end=4.5, y_end=8.8))

show(fig_with_label)

Loading BokehJS ...

### Colorbar

In [103]:
df

,index,Country,Confirmed,conf+100,class,class_
0,0,Algeria,3649.0,3749.0,2,c
1,1,Argentina,4127.0,4227.0,2,b
2,2,Australia,3016.0,3116.0,2,b
3,3,Austria,15357.0,15457.0,0,b
4,4,Bangladesh,6462.0,6562.0,0,a
5,5,Belarus,12208.0,12308.0,0,a
6,6,Belgium,47334.0,47434.0,0,b
7,7,Brazil,73235.0,73335.0,0,c
8,8,Canada,25761.0,25861.0,1,a
9,9,Chile,14365.0,14465.0,0,a


In [104]:
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform

output_notebook()

# data
# map numbers in a range low, high - linearly into a sequence of colors (a palette)
color_mapper = LinearColorMapper(palette='Viridis256', low=df.Confirmed.min(), high=df.Confirmed.max())

# plot
colorbar_fig = figure(plot_width=600, plot_height=400, x_axis_label='Index', y_axis_label='Confermed')
colorbar_fig.circle(x='index', 
                    y='Confirmed',
                    source=df,
                    color=transform('Confirmed', color_mapper), 
                    size=15, 
                    alpha=0.5)

# render a color bar based on a color mapper
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='Confirmed')
colorbar_fig.add_layout(color_bar, 'right')

show(colorbar_fig)

Loading BokehJS ...

### Interactive Widgets

In [115]:
# change size of scatter plot circles
from bokeh.layouts import column
from bokeh.models import Slider

# create figure and plot
change_plot_size = figure(plot_width=600, plot_height=300)
change_plot_size_r = change_plot_size.circle([1,2,3,4,5], [3,2,5,6,4], size=15, alpha=0.5)

# create widget and link
slider = Slider(start=10, end=150, step=5, value=20)
slider.js_link('value', change_plot_size_r.glyph, 'size')

show(column(change_plot_size, slider))

In [127]:
from sklearn import linear_model
from bokeh.layouts import layout
from bokeh.models import Toggle
import numpy as np

# data
x = [1,2,3,4,5,6,7,8,9,10]
X = np.array(x).reshape(-1, 1)
y = [2,2,4,1,5,6,8,2,3,7]
Y = np.array(y).reshape(-1, 1)

# linear regression object
regr = linear_model.LinearRegression()

# fit linear model
regr.fit(X, Y)

# make predictions
pred = regr.predict(X)

# plot with regression line
regr_plot = figure(plot_width=500, plot_height=300, tools='')
regr_plot.scatter(x, y, size=10)
regr_line = regr_plot.line(x, pred.flatten(), line_color='red')

regr_plot.toolbar.logo = None

toggle_button = Toggle(label='line of best fit', button_type='success', active=True)
toggle_button.js_link('active', regr_line, 'visible')


show(layout([regr_plot], [toggle_button]))
#show(regr_plot)